# qual a quantidade média de auto informação por palavra e de informação por mensagem em cada dataset

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
def auto_informacao(i):
    """
    Essa funçãto calcula a autoinformação de um determinado símbolo produzido por uma fonte binária X.
    :param i: Probabilidade de um símbolo ser transmitido pela fonte X. Pode ser um número ou um mapa de :func:`probabilidades`.  
    :return: Informação em bits
    """

    if type(i) != dict:
        return round(-1 * np.log2(i), 4)
    else:
        r = {}
        for key, value in i.items():
            r[key] = round(-1 * np.log2(value), 4)
        return r


## ou informação média
def entropia(c):
    """
    Essa função calcula a entrópia de uma fonte. A entrópia de uma fonte descreve a quantidade média de informação que uma fonte pode transmitir.
    :param c: Mapa de probabilidade de todos os símbolos produzidos por uma fonte binária X. 
    :return: A média ponderada da auto-informacao de todos os símbolos produzidos por uma fonte binária X.
    """
    try:
        c = list(c.values())
    except TypeError:
        raise Exception("c não é um mapa")

    if not c is np.ndarray:
        c = np.array(c)

    return round(sum(np.array(c) * np.log2(1 / np.array(c))), 4)


def probabilidades(c):
    """.value_counts
    Função auxiliar para calcular a probabilidade de ocorrência de cada símbolo de uma coleção.
    :param c: Coleção de símbolos com probabilidade de ocorrência sem dependências.
    :return: Mapa onde as chaves são cada item único da coleção c e os valores são as probabilidades de ocorrência de cada símbolo em relação a todos os outros items.
    """
    try:
        iter(c)
    except TypeError:
        raise Exception("c não é uma lista")

    quantidade_simbolos = {}
    for i in c:
        if quantidade_simbolos.get(i) is None:
            quantidade_simbolos[i] = 1
        else:
            quantidade_simbolos[i] = quantidade_simbolos[i] + 1

    probabilidade_simbolos = {}

    for i in list(set(c)):
        probabilidade_simbolos[i] = quantidade_simbolos[i] / len(c)
    return probabilidade_simbolos

In [18]:
df = pd.read_csv('NoThemeTweets.csv')

In [19]:
print("Analisando cinco tweets:")
print("\n")
df_analise = df[df["tweet_text"].str.len() > 100].sample(5)
for i in range(5):
    mensagem = df_analise.iloc[i].tweet_text
    print(mensagem)
    print("Informação por palavra:")
    probabilidades_por_palavra = probabilidades(mensagem.split())
    informacao_por_palavra = auto_informacao(probabilidades_por_palavra)
    print(informacao_por_palavra)
    max_i = -1
    max_p = None
    for key, value in informacao_por_palavra.items():
        if value > max_i:
            max_i = value
            max_p = key
    print(f"Palavra com mais informação: \"{max_p}\". {max_i} bits/palavra")
    print("Entropia do texto:")
    print(entropia(probabilidades_por_palavra))
    
    print("\n")

Analisando cinco tweets:


Mesmo quando você não está fazendo nada, sua mente não para. São mil pensamentos por segundo que esgotam o seu corpo. Tira um tempo para você, faça com calma algo que você goste, desacelere um pouquinho! Seu corpo e sua mente vão gostar. :)
Informação por palavra:
{'você': 4.4594, ':)': 5.4594, 'para': 5.4594, 'para.': 5.4594, 'desacelere': 5.4594, 'faça': 5.4594, 'corpo': 5.4594, 'o': 5.4594, 'Seu': 5.4594, 'esgotam': 5.4594, 'sua': 4.4594, 'e': 5.4594, 'mil': 5.4594, 'fazendo': 5.4594, 'algo': 5.4594, 'por': 5.4594, 'calma': 5.4594, 'pensamentos': 5.4594, 'seu': 5.4594, 'um': 4.4594, 'corpo.': 5.4594, 'tempo': 5.4594, 'você,': 5.4594, 'mente': 4.4594, 'segundo': 5.4594, 'Tira': 5.4594, 'não': 4.4594, 'pouquinho!': 5.4594, 'com': 5.4594, 'quando': 5.4594, 'nada,': 5.4594, 'está': 5.4594, 'gostar.': 5.4594, 'Mesmo': 5.4594, 'São': 5.4594, 'goste,': 5.4594, 'vão': 5.4594, 'que': 4.4594}
Palavra com mais informação: ":)". 5.4594 bits/palavra
Entropia do texto:


# existe uma variação nessas medidas quando os twittes são neutros, positivos ou negativos?

In [20]:
df = df.sample(20000)
df["entropia"] = df["tweet_text"].apply(lambda x: entropia(probabilidades(x)))
df["entropia"] = df["tweet_text"].apply(lambda x: entropia(probabilidades(x)))

In [21]:
df.groupby("sentiment")["entropia"].mean().sort_values(ascending=False)

sentiment
Positivo    4.186673
Negativo    4.062341
Name: entropia, dtype: float64

In [22]:
df.groupby("sentiment")["entropia"].median().sort_values(ascending=False)

sentiment
Positivo    4.18310
Negativo    4.06005
Name: entropia, dtype: float64

Podemos ver que na media e na mediana a entropia de textos positivos é ligeiramente maior.

# o que é possível afirmar com relação a twiites com temas, usando informação como medida?

Que tweets negativos tendem a ter mais entropia, pois tem palavras mais diversas e buscam passar mais informação.

# se é possível detectar novos usuários usando medidas de informação como parâmetro?

In [38]:
df3 = df[["tweet_date", "entropia"]].sort_values(by="tweet_date").reset_index(drop=True)

In [43]:
df3.describe()

,entropia
count,20000.000000
mean,4.104005
std,0.335038
min,0.710600
25%,3.901275
50%,4.098100
75%,4.331600
max,5.247700
